# **Unigrams: Full Workflow**
The primary purpose of the unigram workflow is to generate a vocabulary **whitelist** that can be used to filter out unwanted words from a multigram corpus. The workflow consists of two steps: (1) downloading the unigram corpus into a database and (2) filtering and normalizing the corpus and generating the whitelist.

## **Setup**
### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from stop_words import get_stop_words
from ngramprep.ngram_filter.lemmatizer import CachedSpacyLemmatizer
from ngramprep.ngram_acquire import download_and_ingest_to_rocksdb
from ngramprep.ngram_filter.pipeline.orchestrator import build_processed_db
from ngramprep.utilities.peek import db_head, db_peek, db_peek_prefix

### Configure
Here we set basic parameters: the corpus to download, the size of the ngrams to download, and the size of the year bins.

In [2]:
db_path_stub = '/scratch/edk202/NLP_corpora/Google_Books/'
archive_path_stub = None
release = '20200217'
language = 'eng'
ngram_size = 1
bin_size = 1

## **Step 1: Download and Ingest**

In [3]:
download_and_ingest_to_rocksdb(
    ngram_size=ngram_size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    archive_path_stub=archive_path_stub,
    ngram_type="tagged",
    overwrite_db=True,
    open_type="write:packed24",
    compact_after_ingest=True
)

N-GRAM ACQUISITION PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2026-01-09 18:59:04

Download Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Ngram repo:           ...//books.storage.googleapis.com/?prefix=ngrams/books/20200217/eng-fiction/1-
DB path:              .../edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1grams.db
File range:           0 to 0
Total files:          1
Files to get:         1
Skipping:             0
Download workers:     1
Batch size:           50,000
Ngram size:           1
Ngram type:           tagged
Overwrite DB:         True
DB Profile:           write:packed24

Download Progress
════════════════════════════════════════════════════════════════════════════════════════════════════


Files Processed: 100%|███████████████████████████████████████████████████████████| 1/1 [02:13<00:00]



Post-Ingestion Compaction
════════════════════════════════════════════════════════════════════════════════════════════════════
Compaction completed in 0:00:20

Processing complete!

Final Summary
════════════════════════════════════════════════════════════════════════════════════════════════════
Fully processed files:       1
Failed files:                0
Total entries written:       2,489,457
Write batches flushed:       1
Uncompressed data processed: 2.78 GB
Processing throughput:       18.37 MB/sec

End Time: 2026-01-09 19:01:39.661829
Total Runtime: 0:02:34.814534
Time per file: 0:02:34.814534
Files per hour: 23.3


 ## **Step 2: Filter, Normalize, and Generate Whitelist**
`config.py` contains generic defaults for the filtering pipeline. You can override these defaults by passing option dictionaries to the `build_processed_db` function, as seen below. By default, we:
1. case-normalize the tokens
2. remove tokens containing non-alphanumeric text
3. remove stopwords using the `stop-words` package
4. lemmatize the tokens using the `spaCy` package—first using a lookup table and falling back to rules when lookups fail
5. Create a whitelist of the top 20,000 most frequent words that pass a `pyenchant` spell-check and appear in all corpora from 1900–2019 (inclusive).

In [ ]:
filter_options = {
    'stop_set': set(get_stop_words("english")),
    'lemma_gen': CachedSpacyLemmatizer(language="en"),
    'min_context_tokens': 1,
    'bin_size': bin_size
}

whitelist_options = {
    'output_whitelist_top_n': 30_000,
    'output_whitelist_year_range': (1900, 2019),
    'output_whitelist_spell_check': True,
    'output_whitelist_spell_check_language': "en"
}

build_processed_db(
    mode="restart",
    ngram_size=ngram_size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    num_workers=20,
    num_initial_work_units=300,
    cache_partitions=True,
    use_cached_partitions=False,
    progress_every_s=5,
    compact_after_ingest=True,
    **filter_options,
    **whitelist_options
);

TypeError: build_processed_db() got an unexpected keyword argument 'min_len'

## **Optional: Inspect Database Files**

### `db_head`: Show first N records

In [9]:
db = f'{db_path_stub}{release}/{language}/{ngram_size}gram_files/{ngram_size}grams_processed.db'

db_head(db, n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   FALSE
     Value: Total: 124,503 occurrences in 110,465 volumes (1538-2019, 380 bins)

[ 2] Key:   TRUE
     Value: Total: 4,241,065 occurrences in 2,916,658 volumes (1501-2019, 440 bins)

[ 3] Key:   aa
     Value: Total: 16,857,095 occurrences in 4,813,142 volumes (1475-2019, 467 bins)

[ 4] Key:   aaa
     Value: Total: 4,449,225 occurrences in 1,052,377 volumes (1477-2019, 402 bins)

[ 5] Key:   aaaa
     Value: Total: 472,912 occurrences in 91,764 volumes (1477-2019, 337 bins)



### `db_peek`: Show records starting from a key

In [10]:
db = f'{db_path_stub}{release}/{language}/{ngram_size}gram_files/{ngram_size}grams_processed.db'

db_peek(db, start_key="à", n=5)

5 key-value pairs starting from c3a0:
────────────────────────────────────────────────────────────────────────────────────────────────────


### `db_peek_prefix`: Show records matching a prefix

In [11]:
db = f'{db_path_stub}{release}/{language}/{ngram_size}gram_files/{ngram_size}grams_processed.db'

db_peek_prefix(db, prefix=b"tarnation", n=5)

5 key-value pairs with prefix 7461726e6174696f6e:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   tarnation
     Value: Total: 67,827 occurrences in 48,385 volumes (1633-2019, 260 bins)

[ 2] Key:   tarnational
     Value: Total: 114 occurrences in 92 volumes (1913-2010, 53 bins)

[ 3] Key:   tarnationest
     Value: Total: 53 occurrences in 53 volumes (1840-2019, 30 bins)

[ 4] Key:   tarnationly
     Value: Total: 94 occurrences in 94 volumes (1833-2019, 44 bins)

[ 5] Key:   tarnations
     Value: Total: 1,189 occurrences in 1,048 volumes (1606-2019, 198 bins)

